In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import os
import duckdb
from datetime import datetime
import glob
import multiprocessing as mp
from functools import partial
import time
from scipy import stats


In [2]:
def calculate_daily_factor(trading_date):

    date_str = pd.to_datetime(trading_date).strftime('%Y%m%d')
    order_pth = f"/data/cephfs/order/{date_str}.parquet"

    if not os.path.exists(order_pth):
        raise FileNotFoundError(f"数据文件不存在: {order_pth}")

    # 为每个进程创建独立的duckdb连接
    conn = duckdb.connect(database=':memory:')

    # 定义六个时间段的查询
    time_intervals = {
        'pre_open_915_920': "order_time >= 91500000 AND order_time < 92000000",
        'pre_open_920_925': "order_time >= 92000000 AND order_time < 92500000",
        'early_930_1000': "order_time >= 93000000 AND order_time < 100000000",
        'main_1000_1430': "order_time >= 100000000 AND order_time < 143000000",
        'late_1430_1457': "order_time >= 143000000 AND order_time < 145700000",
        'close_1457_1500': "order_time >= 145700000 AND order_time < 150000000",
        'comtinue_930_1457':"order_time >= 93000000 AND order_time < 145700000"
    }
    
    # 创建字典来存储不同时间间隔的查询结果
    bid_dfs = {}
    ask_dfs = {}

    try:
        # 为每个时间段和买卖方向创建临时表
        for interval_name, time_condition in time_intervals.items():
            # 买方订单
            bid_query = f"""
            SELECT 
                security_code,
                order_side, 
                order_type,
                order_details,
                order_price,
                order_price_adj,
                order_volume,
                order_time
            FROM '{order_pth}'
            WHERE order_type = 'A'
                AND order_details = 'L'
                AND order_side = 1
                AND {time_condition}
            """
            
            # 卖方订单
            ask_query = f"""
            SELECT 
                security_code,
                order_side, 
                order_type,
                order_details,
                order_price,
                order_price_adj,
                order_volume,
                order_time
            FROM '{order_pth}'
            WHERE order_type = 'A'
                AND order_details = 'L'
                AND order_side = -1
                AND {time_condition}
            """
            
            # 执行查询并存储结果
            bid_dfs[f"bid_{interval_name}"] = conn.execute(bid_query).fetchdf()
            ask_dfs[f"ask_{interval_name}"] = conn.execute(ask_query).fetchdf()
        
        # 定义计算按交易量加权的中位数价格的函数
        def volume_weighted_median(group):
            if len(group) == 0:
                return np.nan
                
            # 按价格排序
            sorted_group = group.sort_values('order_price')
            
            # 计算总交易量
            total_volume = sorted_group['order_volume'].sum()
            if total_volume == 0:
                return np.nan
                
            # 计算累计交易量
            sorted_group['cumulative_volume'] = sorted_group['order_volume'].cumsum()
            
            # 找到累计交易量超过总量50%的第一个价格
            median_price = sorted_group[sorted_group['cumulative_volume'] >= total_volume * 0.5]['order_price'].iloc[0]
            median_volume = sorted_group[sorted_group['cumulative_volume'] >= total_volume * 0.5]['order_volume'].iloc[0]
            return median_price,median_volume
        
        # 定义计算按交易量加权的MAD的函数
        def volume_weighted_mad(group):
            if len(group) < 2:  # 至少需要2个数据点才能计算MAD
                return np.nan
                
            # 计算按交易量加权的中位数价格
            median_price, median_volume = volume_weighted_median(group)
            
            # 计算每个价格与中位数的绝对偏差
            group['abs_deviation'] = np.abs(group['order_price'] - median_price)
            
            # 按绝对偏差排序
            sorted_by_dev = group.sort_values('abs_deviation')
            
            # 计算总交易量
            total_volume = sorted_by_dev['order_volume'].sum()
            if total_volume == 0:
                return np.nan
                
            # 计算累计交易量
            sorted_by_dev['cumulative_volume'] = sorted_by_dev['order_volume'].cumsum()
            
            # 找到累计交易量超过总量50%的第一个绝对偏差
            mad = sorted_by_dev[sorted_by_dev['cumulative_volume'] >= (total_volume - median_volume) * 0.5]['abs_deviation'].iloc[0]
            
            return mad
        
        bid_mad_results = {}
        ask_mad_results = {}
        combined_mad_results = {}

        for interval_name in time_intervals.keys():
            # 获取当前时间区间的买卖订单数据
            bid_df = bid_dfs[f"bid_{interval_name}"]
            ask_df = ask_dfs[f"ask_{interval_name}"]
            
            # 计时开始
            start_time = time.time()
            
            # 1. 计算买方订单价格的MAD
            bid_mad = bid_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
            bid_mad.columns = ['security_code', f'bid_mad_{interval_name}']
            bid_mad_results[interval_name] = bid_mad
            
            # 2. 计算卖方订单价格的MAD
            ask_mad = ask_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
            ask_mad.columns = ['security_code', f'ask_mad_{interval_name}']
            ask_mad_results[interval_name] = ask_mad
            
            # 3. 合并买卖订单后计算MAD
            # 创建一个合并的DataFrame，添加'direction'列来标识买卖方向
            bid_df_copy = bid_df.copy()
            ask_df_copy = ask_df.copy()
            bid_df_copy['direction'] = 'bid'
            ask_df_copy['direction'] = 'ask'
            
            combined_df = pd.concat([bid_df_copy, ask_df_copy])
            
            # 对合并后的数据计算MAD
            combined_mad = combined_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
            combined_mad.columns = ['security_code', f'combined_mad_{interval_name}']
            combined_mad_results[interval_name] = combined_mad
            
            # 计时结束
            end_time = time.time()
            print(f"{trading_date}: Time interval {interval_name} processed in {end_time - start_time:.4f} seconds")

        # 创建结果DataFrame，从第一个时间区间开始
        first_interval = list(time_intervals.keys())[0]
        result_df = bid_mad_results[first_interval]
        
        # 合并所有结果
        for interval_name in time_intervals.keys():
            # 合并买方MAD（跳过第一个，因为已经用它初始化了result_df）
            if interval_name != first_interval:
                result_df = pd.merge(result_df, bid_mad_results[interval_name], 
                                    on='security_code', how='outer')
            
            # 合并卖方MAD
            result_df = pd.merge(result_df, ask_mad_results[interval_name], 
                                on='security_code', how='outer')
            
            # 合并组合MAD
            result_df = pd.merge(result_df, combined_mad_results[interval_name], 
                                on='security_code', how='outer')
    finally:
        conn.close()
   
    # 添加日期列
    result_df['date'] = pd.to_datetime(trading_date)
    # 添加因子名称标识
    # result_df['factor_name'] = "six_itvs_order_price_volume_weighted_MAD"
    
    return result_df


In [3]:
def process_single_date(trading_date, factor_name):
    
    """处理单个交易日的函数"""
   
    date_str = trading_date.strftime('%Y%m%d')
    output_dir = f"./factors/{factor_name}"
    file_path = f"{output_dir}/{date_str}.parquet"
    
    # 如果文件已存在，跳过处理
    if os.path.exists(file_path):
        print(f"文件已存在，跳过: {date_str}")
        return date_str, True, "已存在"
    
    # 检查原始数据文件是否存在
    data_file = f"/data/cephfs/order/{date_str}.parquet"
    if not os.path.exists(data_file):
        print(f"跳过日期 {date_str}: 原始数据文件不存在")
        return date_str, False, "数据文件不存在"
    
    try:
        daily_factor_df = calculate_daily_factor(trading_date)
        
        # daily_factor_df['date'] = date_str

        daily_factor_df.to_parquet(file_path, index=False)
        print(f"已生成因子文件: {file_path}")
        return date_str, True, "成功"
        
    except Exception as e:
        error_msg = str(e)
        print(f"处理日期 {date_str} 时出错: {error_msg}")
        return date_str, False, error_msg


def derive_daily_factor(start_date, end_date, factor_name, num_processes=20):
    start_time = time.time()
    
    trading_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 创建输出目录
    output_dir = f"./factors/{factor_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建进程池
    pool = mp.Pool(processes=num_processes)
    
    # 创建带有固定参数的处理函数
    process_date_with_args = partial(process_single_date, factor_name=factor_name)
    
    # 提交所有任务到进程池并获取结果
    print(f"开始使用 {num_processes} 个进程并行处理 {len(trading_dates)} 个交易日...")
    results = pool.map(process_date_with_args, trading_dates)
    
    # 关闭进程池
    pool.close()
    pool.join()
    
    # 分析处理结果
    processed_dates = [date_str for date_str, success, _ in results if success]
    skipped_dates = [date_str for date_str, success, _ in results if not success]
    
    # 打印处理结果摘要
    print(f"\n处理完成:")
    print(f"成功处理 {len(processed_dates)} 个日期")
    print(f"跳过 {len(skipped_dates)} 个日期")

    return output_dir


In [ ]:
def main():
    start_date = '2016-06-20'
    end_date = '2025-05-31'
    factor_name = 'six_itvs_order_price_volume_weighted_MAD'
    factor_dir = derive_daily_factor (start_date, end_date, factor_name)

main()

跳过日期 20160731: 原始数据文件不存在跳过日期 20170514: 原始数据文件不存在跳过日期 20170403: 原始数据文件不存在跳过日期 20170624: 原始数据文件不存在跳过日期 20160910: 原始数据文件不存在跳过日期 20180225: 原始数据文件不存在
跳过日期 20180407: 原始数据文件不存在




跳过日期 20170625: 原始数据文件不存在跳过日期 20170404: 原始数据文件不存在


跳过日期 20180408: 原始数据文件不存在跳过日期 20160911: 原始数据文件不存在

开始使用 20 个进程并行处理 3268 个交易日...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

/tmp/ipykernel_3954866/940483840.py:136: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ask_mad = ask_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
/tmp/ipykernel_3954866/940483840.py:136: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ask_mad = ask_df.groupby('security_code').apply(volume_weighted_mad).reset_index()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

/tmp/ipykernel_3954866/940483840.py:131: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bid_mad = bid_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
/tmp/ipykernel_3954866/940483840.py:131: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bid_mad = bid_df.groupby('security_code').apply(volume_weighted_mad).reset_index()
/tmp/ipykernel_3954866/940483840.py:131: DeprecationWarning: DataFrame

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [16]:
def filter_and_pivot_factor(all_factor_data, factor_name):
    """
    筛选特定因子列且该列不为NaN的数据，并进行透视
    
    参数:
    all_factor_data: 原始数据DataFrame
    factor_name: 要筛选的因子列名称，例如'bid_pre_open_915_920_p01_p99'
    
    返回:
    透视后的DataFrame，索引为date，列为security_code
    """
    # 步骤1: 筛选必要的列
    filtered_data = all_factor_data[['date', 'security_code', factor_name]]
    
    # 步骤2: 去除因子列为NaN的行
    filtered_data = filtered_data.dropna(subset=[factor_name])
    
    # 步骤3: 检查是否存在重复的(date, security_code)组合
    duplicates = filtered_data.duplicated(subset=['date', 'security_code'], keep=False)
    if duplicates.any():
        print(f"警告: 发现{duplicates.sum()}条重复记录，将保留第一条")
        # 如果有重复，保留第一条
        filtered_data = filtered_data.drop_duplicates(subset=['date', 'security_code'], keep='first')

    output_path = f"/data/home/lexuanchen/Factors/Order/Signal/Raw_Order_Volume_Price_MAD"
    os.makedirs(output_path, exist_ok=True)
    output_file_path = f"{output_path}/Raw_{factor_name}.csv"
    filtered_data.to_csv(output_file_path)
    print(f"已成功保存因子原文件：{factor_name}")
    
    # 步骤4: 透视表，将security_code作为列，date作为索引
    pivot_data = filtered_data.pivot(index='date', columns='security_code', values=factor_name)
    
    return pivot_data


In [17]:
def calculate_rolling_20d_avg(factor_names, factor_dir, start_date=None, end_date=None, min_window=5):
    
    print(factor_names)
    
    if not isinstance(factor_names, list):
        factor_names = [factor_names]
    
    parquet_pattern = os.path.join(factor_dir, "*.parquet")
    parquet_files = glob.glob(parquet_pattern)
    print(f"在目录 {factor_dir} 中找到 {len(parquet_files)} 个Parquet文件")
    

    conn = duckdb.connect(database=':memory:')
    all_factor_data = conn.execute(f"""
        SELECT * FROM read_parquet('{parquet_pattern}')
    """).fetchdf()
    conn.close()
    
    

    missing_factors = [f for f in factor_names if f not in all_factor_data.columns]
    if missing_factors:
        raise ValueError(f"在Parquet文件中未找到以下因子列: {', '.join(missing_factors)}")
    
    all_factor_data['date'] = pd.to_datetime(all_factor_data['date'])

    all_factor_data = all_factor_data.sort_values(['date', 'security_code'])
    
    # 筛选日期范围
    if start_date:
        start_date = pd.to_datetime(start_date)
        all_factor_data = all_factor_data[all_factor_data['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        all_factor_data = all_factor_data[all_factor_data['date'] <= end_date]

    # 获取所有个股代码
    all_securities = all_factor_data['security_code'].unique()

    result_df = all_factor_data[['date', 'security_code']].copy()

    # 为每个因子计算滚动平均
    for factor_name in factor_names:
        print(f"\n处理因子: {factor_name}")
        
        #转置成宽表
        pivot_data = filter_and_pivot_factor(all_factor_data,factor_name)

        # 对宽表直接应用rolling
        rolling_avg = pivot_data.rolling(window=20, min_periods=min_window).mean()

        # 将结果转换回长格式
        factor_df = rolling_avg.stack().reset_index()
        factor_df.columns = ['date', 'security_code', factor_name]
        # 推测为反向因子
        # factor_df[factor_name] = - factor_df[factor_name]

        # 计算每日因子覆盖率
        # 计算每个日期非NaN的因子值数量
        non_nan_counts = factor_df.dropna(subset=[factor_name]).groupby('date').size()

        # 计算覆盖率
        coverage = non_nan_counts / len(all_securities)
        print(f"\n{factor_name}因子覆盖率统计: 平均={coverage.mean():.2f}, 最小={coverage.min():.2f}")

        result_df = pd.merge(
            result_df, 
            factor_df,
            on=['date', 'security_code'],
            how='left'
        )
    
    return result_df

In [18]:
def main():
    start_date = '2019-01-01'
    end_date = '2025-05-31'
    
    # 定义时间段列表
    time_intervals = [
        'pre_open_915_920',
        'pre_open_920_925',
        'early_930_1000',
        'main_1000_1430',
        'late_1430_1457',
        'close_1457_1500',
        'comtinue_930_1457'
    ]
    
    # 定义因子类型
    factor_types = [
        'bid_mad',
        'ask_mad',
        'combined_mad'
    ]
    
    # 生成所有因子名称列表
    all_factors = []
    for factor_type in factor_types:
        for interval in time_intervals:
            all_factors.append(f"{factor_type}_{interval}")
    
    # 为每种因子类型创建单独的目录
    factor_dirs = {}
    for factor_type in factor_types:

        factor_dir = f"./factors/six_itvs_order_price_volume_weighted_MAD"
        factor_dirs[factor_type] = factor_dir

        
        # 统一化列名
    # standardize_parquet_files(factor_dir)

    output_path = f"/data/home/lexuanchen/Factors/Order/Signal/Six_Itvs_Order_Price_VolumeWeighted_MAD"
    os.makedirs(output_path, exist_ok=True)

    # 为每种因子类型计算滚动平均值
    for factor_type, factor_dir in factor_dirs.items():
        # 获取该类型的因子列表
        type_factors = [f for f in all_factors if f.startswith(factor_type.split('_')[0])]
        
        # 计算滚动平均
        result_df = calculate_rolling_20d_avg(type_factors, factor_dir, start_date, end_date, min_window=5)
        
        print(f"{factor_type} 因子共计 {len(result_df)} 条记录")
        print("\n数据预览:")
        print(result_df.head())
        
        # 保存每个因子的CSV文件
        for factor in type_factors:
            output_file_path = f"{output_path}/20d_{factor}.csv"
            factor_df = result_df[['date', 'security_code', factor]].dropna(subset=[factor])
            factor_df.to_csv(output_file_path, index=False)

# 执行主函数
if __name__ == "__main__":
    main()


['bid_mad_pre_open_915_920', 'bid_mad_pre_open_920_925', 'bid_mad_early_930_1000', 'bid_mad_main_1000_1430', 'bid_mad_late_1430_1457', 'bid_mad_close_1457_1500', 'bid_mad_comtinue_930_1457']
在目录 ./factors/six_itvs_order_price_volume_weighted_MAD 中找到 1549 个Parquet文件


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: bid_mad_pre_open_915_920
已成功保存因子原文件：bid_mad_pre_open_915_920

bid_mad_pre_open_915_920因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: bid_mad_pre_open_920_925
已成功保存因子原文件：bid_mad_pre_open_920_925

bid_mad_pre_open_920_925因子覆盖率统计: 平均=0.84, 最小=0.58

处理因子: bid_mad_early_930_1000
已成功保存因子原文件：bid_mad_early_930_1000

bid_mad_early_930_1000因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: bid_mad_main_1000_1430
已成功保存因子原文件：bid_mad_main_1000_1430

bid_mad_main_1000_1430因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: bid_mad_late_1430_1457
已成功保存因子原文件：bid_mad_late_1430_1457

bid_mad_late_1430_1457因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: bid_mad_close_1457_1500
已成功保存因子原文件：bid_mad_close_1457_1500

bid_mad_close_1457_1500因子覆盖率统计: 平均=0.84, 最小=0.63

处理因子: bid_mad_comtinue_930_1457
已成功保存因子原文件：bid_mad_comtinue_930_1457

bid_mad_comtinue_930_1457因子覆盖率统计: 平均=0.84, 最小=0.66
bid_mad 因子共计 6955019 条记录

数据预览:
        date security_code  bid_mad_pre_open_915_920  \
0 2019-01-02        000001                       NaN   
1 2019-01-02        000002               

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: ask_mad_pre_open_915_920
已成功保存因子原文件：ask_mad_pre_open_915_920

ask_mad_pre_open_915_920因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: ask_mad_pre_open_920_925
已成功保存因子原文件：ask_mad_pre_open_920_925

ask_mad_pre_open_920_925因子覆盖率统计: 平均=0.84, 最小=0.64

处理因子: ask_mad_early_930_1000
已成功保存因子原文件：ask_mad_early_930_1000

ask_mad_early_930_1000因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: ask_mad_main_1000_1430
已成功保存因子原文件：ask_mad_main_1000_1430

ask_mad_main_1000_1430因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: ask_mad_late_1430_1457
已成功保存因子原文件：ask_mad_late_1430_1457

ask_mad_late_1430_1457因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: ask_mad_close_1457_1500
已成功保存因子原文件：ask_mad_close_1457_1500

ask_mad_close_1457_1500因子覆盖率统计: 平均=0.84, 最小=0.64

处理因子: ask_mad_comtinue_930_1457
已成功保存因子原文件：ask_mad_comtinue_930_1457

ask_mad_comtinue_930_1457因子覆盖率统计: 平均=0.84, 最小=0.66
ask_mad 因子共计 6955019 条记录

数据预览:
        date security_code  ask_mad_pre_open_915_920  \
0 2019-01-02        000001                       NaN   
1 2019-01-02        000002               

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


处理因子: combined_mad_pre_open_915_920
已成功保存因子原文件：combined_mad_pre_open_915_920

combined_mad_pre_open_915_920因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_pre_open_920_925
已成功保存因子原文件：combined_mad_pre_open_920_925

combined_mad_pre_open_920_925因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_early_930_1000
已成功保存因子原文件：combined_mad_early_930_1000

combined_mad_early_930_1000因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_main_1000_1430
已成功保存因子原文件：combined_mad_main_1000_1430

combined_mad_main_1000_1430因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_late_1430_1457
已成功保存因子原文件：combined_mad_late_1430_1457

combined_mad_late_1430_1457因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_close_1457_1500
已成功保存因子原文件：combined_mad_close_1457_1500

combined_mad_close_1457_1500因子覆盖率统计: 平均=0.84, 最小=0.66

处理因子: combined_mad_comtinue_930_1457
已成功保存因子原文件：combined_mad_comtinue_930_1457

combined_mad_comtinue_930_1457因子覆盖率统计: 平均=0.84, 最小=0.66
combined_mad 因子共计 6955019 条记录

数据预览:
        date security_code  combined_mad_pr